In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import random
import plotly.express as px
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
from scipy.stats.contingency import relative_risk

https://docs.scipy.org/doc/scipy/reference/stats.contingency.html

https://docs.scipy.org/doc/scipy/reference/stats.mstats.html

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.contingency.relative_risk.html#scipy.stats.contingency.relative_risk

Anteilswerttest

sachen to do: schnellere methode für den diff abgleich finden
              
              schauen ob andere test dafür geiegnet sind

              geomap rumprobieren

In [2]:
df = pd.read_parquet("Shopping Carts.parquet")
df.reset_index(inplace=True)

In [3]:
df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index()
df = pd.merge(df,df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index())

In [4]:
df

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


### regionale unterschiede


In [5]:
df_regional = df.groupby("county").agg(count = ("order_id","nunique")).reset_index()

In [6]:
df_regional = df_regional.sort_values("count")

In [14]:
exceptions = df_regional.head(3).county.unique()

In [23]:
df_regional.tail(10)

,county,count
42,Santa Cruz,19340
20,Mariposa,23234
25,Monterey,24142
54,Ventura,24684
49,Sutter,25420
13,Kern,31601
8,Fresno,32685
28,Orange,34496
9,Glenn,35541
3,Calaveras,43559


große diskrepanz in den ausleihzahlen besonder San Bernardino, Placer, Trinity als problemfall

dazu map

#### top produkte und aisle

In [8]:
df_products = df.groupby("product_name").agg(coun = ("product_name","count")).reset_index().sort_values("coun",ascending=False)

In [31]:
df_products.tail(50)

,product_name,coun
32797,Pork Shoulder,1
10315,Creamy Supreme Frosting Chocolate,1
4711,Body Powder Original Strength,1
41094,The Amazing Prune Light Prune Juice Cocktail,1
19176,Imperial Champagne,1
10285,Creamy Oat Bran Hot Cereal,1
19119,"Iced Tea, Lemon",1
19117,Iced Tea with Raspberry,1
19027,Ice Cream Cake Celebration,1
19047,Ice Cream Super Chunky Cookie Dough,1


In [9]:
df_aisle = df.groupby("aisle").agg(coun = ("aisle","count")).reset_index().sort_values("coun",ascending=False)

In [10]:
df_aisle

,aisle,coun
50,fresh fruits,679534
53,fresh vegetables,639485
98,packaged vegetables fruits,333355
133,yogurt,274700
93,packaged cheese,184845
...,...,...
44,eye ear care,1744
3,baby bath body care,1686
2,baby accessories,1463
8,beauty,1195


#### oder_size je county

In [76]:
df

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


In [79]:
df_order_size = df.groupby(["order_id","county"])[["order_size"]].max().reset_index()

In [83]:
df_order_size = df_order_size.groupby("county")[["order_size"]].median().reset_index()

In [86]:
df_order_size.sort_values("order_size")

,county,order_size
0,Alameda,3.0
10,Humboldt,3.0
34,San Bernardino,3.0
29,Placer,3.5
4,Colusa,5.0
21,Mendocino,5.0
26,Napa,5.0
2,Amador,5.0
48,Stanislaus,6.0
47,Sonoma,6.0


#### top aisle pro county


In [7]:
df_3 = df.groupby(["county","aisle"]).agg(anzahl=("aisle","count")).reset_index()

In [8]:
counties = df_3.county.unique().tolist()
for i in counties:
    df_3.loc[df_3["county"] == i,"rank"] = df_3.loc[df_3["county"] == i].anzahl.rank(method="min",ascending=False)

In [37]:
df_3

,county,aisle,anzahl,rank
0,Alameda,air fresheners candles,6,115.0
1,Alameda,asian foods,10,107.0
2,Alameda,baby accessories,1,126.0
3,Alameda,baby bath body care,1,126.0
4,Alameda,baby food formula,29,77.0
...,...,...,...,...
7336,Yuba,trash bags liners,187,97.0
7337,Yuba,vitamins supplements,314,80.0
7338,Yuba,water seltzer sparkling water,7977,3.0
7339,Yuba,white wines,375,71.0


#### top 10 aisled pro county

In [13]:
df_4 = df_3[df_3["rank"].isin(range(11))]
df_4 = df_4.sort_values(["county","rank"])

In [14]:
df_4

,county,aisle,anzahl,rank
49,Alameda,fresh fruits,2824,1.0
52,Alameda,fresh vegetables,982,2.0
132,Alameda,yogurt,933,3.0
130,Alameda,water seltzer sparkling water,821,4.0
97,Alameda,packaged vegetables fruits,610,5.0
...,...,...,...,...
7290,Yuba,milk,4686,6.0
7232,Yuba,chips pretzels,4492,7.0
7249,Yuba,energy granola bars,3931,8.0
7324,Yuba,soft drinks,3830,9.0


In [40]:
df_3

,county,aisle,anzahl,rank
0,Alameda,air fresheners candles,6,115.0
1,Alameda,asian foods,10,107.0
2,Alameda,baby accessories,1,126.0
3,Alameda,baby bath body care,1,126.0
4,Alameda,baby food formula,29,77.0
...,...,...,...,...
7336,Yuba,trash bags liners,187,97.0
7337,Yuba,vitamins supplements,314,80.0
7338,Yuba,water seltzer sparkling water,7977,3.0
7339,Yuba,white wines,375,71.0


#### frage wegen mengen bias (weil counties ja unterschiedliche ausleihzeilen haben dann den durchschnitt von je county nehmen oder den nochmal auf gesmatmenge projezieren)

In [9]:
df_5 = df_3.groupby("aisle").agg(rank_mean=("rank","mean")).reset_index()

In [10]:
df_6 = pd.merge(df_3,df_5)

In [11]:
df_8 = df_6.loc[(df_6["rank"] > df_6.rank_mean * 1.5)]

In [213]:
df_8

,county,aisle,anzahl,rank,rank_mean
55,Alameda,asian foods,10,107.0,52.290909
81,Napa,asian foods,25,93.0,52.290909
211,Alameda,baby food formula,29,77.0,34.600000
217,Del Norte,baby food formula,67,73.0,34.600000
221,Humboldt,baby food formula,9,83.0,34.600000
...,...,...,...,...,...
7161,Sacramento,water seltzer sparkling water,1423,14.0,8.859649
7165,San Francisco,water seltzer sparkling water,438,63.0,8.859649
7166,San Joaquin,water seltzer sparkling water,59,40.0,8.859649
7245,Humboldt,yogurt,203,8.0,4.018182


In [15]:
df_8[df_8.county.isin(exceptions)]

,county,aisle,anzahl,rank,rank_mean
1301,Trinity,cereal,1,75.0,22.854545
1358,Trinity,chips pretzels,9,26.0,9.719298
1690,Trinity,cookies cakes,1,75.0,39.666667
1730,San Bernardino,crackers,1,30.0,18.947368
1747,Trinity,crackers,8,30.0,18.947368
2230,San Bernardino,eggs,1,30.0,17.771930
2302,Trinity,energy granola bars,1,75.0,22.854545
2725,San Bernardino,fresh fruits,5,11.0,1.508772
2888,Placer,fresh vegetables,8,6.0,2.315789
2893,San Bernardino,fresh vegetables,3,19.0,2.315789


In [12]:
df_9 = df_6.loc[(df_6["rank"] < df_6.rank_mean * 0.5)]

In [20]:
df_9

,county,aisle,anzahl,rank,rank_mean
29,Placer,air fresheners candles,1,36.0,108.600000
104,Trinity,asian foods,10,25.0,52.290909
212,Alpine,baby food formula,2647,9.0,34.600000
219,Fresno,baby food formula,5425,12.0,34.600000
220,Glenn,baby food formula,6390,13.0,34.600000
...,...,...,...,...,...
7231,Trinity,white wines,7,32.0,97.693878
7277,Sierra,yogurt,1567,2.0,4.018182
7283,Tehama,yogurt,718,1.0,4.018182
7289,Yuba,yogurt,8243,2.0,4.018182


In [21]:
df_9[df_9.county.isin(exceptions)]

,county,aisle,anzahl,rank,rank_mean
29,Placer,air fresheners candles,1,36.0,108.600000
104,Trinity,asian foods,10,25.0,52.290909
510,Placer,beers coolers,2,26.0,84.634615
515,San Bernardino,beers coolers,8,4.0,84.634615
567,San Bernardino,body lotions soap,4,13.0,85.836364
...,...,...,...,...,...
6619,Placer,specialty wines champagnes,1,36.0,112.854167
6723,Placer,spirits,3,20.0,95.979592
6728,San Bernardino,spirits,2,24.0,95.979592
6833,San Bernardino,tea,6,6.0,38.803571


In [59]:
df_9.columns

Index(['county', 'aisle', 'anzahl', 'rank', 'rank_mean'], dtype='object')

In [13]:
df_2 = df.copy()
relevant = []
for i in df_9.values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county"))
    if p < 0.05:
        relevant.append(i)




In [16]:
df_test

,county,aisle,not_aisle
0,Alpine,3290,6046094
1,Amador,3290,6046094
2,Calaveras,3290,6046094
3,Colusa,135,34808
4,Contra Costa,3290,6046094
5,Del Norte,3290,6046094
6,El Dorado,3290,6046094
7,Fresno,3290,6046094
8,Glenn,3290,6046094
9,Imperial,3290,6046094


In [15]:
relevant.__len__()

200

In [37]:
df_8

,county,aisle,anzahl,rank,rank_mean
55,Alameda,asian foods,10,107.0,52.290909
81,Napa,asian foods,25,93.0,52.290909
211,Alameda,baby food formula,29,77.0,34.600000
217,Del Norte,baby food formula,67,73.0,34.600000
221,Humboldt,baby food formula,9,83.0,34.600000
...,...,...,...,...,...
7161,Sacramento,water seltzer sparkling water,1423,14.0,8.859649
7165,San Francisco,water seltzer sparkling water,438,63.0,8.859649
7166,San Joaquin,water seltzer sparkling water,59,40.0,8.859649
7245,Humboldt,yogurt,203,8.0,4.018182


In [38]:
df_2 = df.copy()
relevant2 = []
for i in df_8.values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == "Alameda"))[-1]) | (df_test.county ==i[0]) ].set_index("county"))
    if p < 0.05:
        relevant2.append(i)


In [142]:
np.delete(df_test.county.unique(),np.where(df_test.county.unique() == "Alameda"))[-1]

'Yuba'

In [40]:
relevant2.__len__()

284

In [25]:
df_tes4 = pd.DataFrame(relevant,columns=df_9.columns)

In [196]:
df_9

,county,aisle,anzahl,rank,rank_mean
29,Placer,air fresheners candles,1,36.0,108.600000
104,Trinity,asian foods,10,25.0,52.290909
212,Alpine,baby food formula,2647,9.0,34.600000
219,Fresno,baby food formula,5425,12.0,34.600000
220,Glenn,baby food formula,6390,13.0,34.600000
...,...,...,...,...,...
7231,Trinity,white wines,7,32.0,97.693878
7277,Sierra,yogurt,1567,2.0,4.018182
7283,Tehama,yogurt,718,1.0,4.018182
7289,Yuba,yogurt,8243,2.0,4.018182


In [22]:
df_2 = df.copy()
relevant3 = []
for i in df_9.values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    
    df_test3 = df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county")
    result = relative_risk(df_test3[df_test3.index == i[0]].values[0][0],df_test3[df_test3.index == i[0]].values[0][1]+df_test3[df_test3.index == i[0]].values[0][0],df_test3[df_test3.index != i[0]].values[0][0],df_test3[df_test3.index != i[0]].values[0][1]+df_test3[df_test3.index != i[0]].values[0][0])
    if result.confidence_interval(confidence_level=0.95)[0] > 1:
                relevant3.append(i)
  

        

In [24]:
relevant3.__len__()

205

In [42]:
df_2 = df.copy()
relevant4 = []
for i in df_8.values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    
    df_test3 = df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county")
    result = relative_risk(df_test3[df_test3.index == i[0]].values[0][0],df_test3[df_test3.index == i[0]].values[0][1]+df_test3[df_test3.index == i[0]].values[0][0],df_test3[df_test3.index != i[0]].values[0][0],df_test3[df_test3.index != i[0]].values[0][1]+df_test3[df_test3.index != i[0]].values[0][0])
    if result.confidence_interval(confidence_level=0.99)[0] > 1 or result.confidence_interval(confidence_level=0.99)[1] < 1:
                relevant4.append(i)

In [44]:
relevant4.__len__()

289

In [26]:
df_tes5 = pd.DataFrame(relevant3,columns=df_9.columns)

In [27]:
df_tes5

,county,aisle,anzahl,rank,rank_mean
0,Alpine,baby food formula,2647,9.0,34.600000
1,Fresno,baby food formula,5425,12.0,34.600000
2,Glenn,baby food formula,6390,13.0,34.600000
3,Nevada,baby food formula,835,15.0,34.600000
4,Orange,baby food formula,6300,13.0,34.600000
...,...,...,...,...,...
200,Trinity,white wines,7,32.0,97.693878
201,Sierra,yogurt,1567,2.0,4.018182
202,Tehama,yogurt,718,1.0,4.018182
203,Yuba,yogurt,8243,2.0,4.018182


In [32]:
pd.concat([df_tes4,df_tes5]).drop_duplicates(keep=False)

,county,aisle,anzahl,rank,rank_mean
83,Alpine,fresh vegetables,13096,1.0,2.315789
86,Lassen,fresh vegetables,1850,1.0,2.315789
88,Riverside,fresh vegetables,4740,1.0,2.315789
90,San Luis Obispo,fresh vegetables,3259,1.0,2.315789
93,Santa Clara,fresh vegetables,8899,1.0,2.315789
18,Placer,beers coolers,2,26.0,84.634615
24,San Bernardino,bread,6,6.0,12.315789
35,San Bernardino,candy chocolate,4,13.0,31.000000
53,San Bernardino,cocoa drink mixes,1,30.0,105.833333
55,San Bernardino,coffee,3,19.0,43.381818


In [33]:
df_2 = df.copy()
relevant = []
for i in pd.concat([df_tes4,df_tes5]).drop_duplicates(keep=False).values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county"))
    print(i[0],p)

Alpine 3.7622803811415533e-13
Lassen 1.9065650145223485e-34
Riverside 0.0005215719198080733
San Luis Obispo 3.6384865940011096e-10
Santa Clara 3.734382857165846e-28
Placer 0.06790247276278678
San Bernardino 0.07765923797341606
San Bernardino 0.06458071569645682
San Bernardino 0.21393905570026148
San Bernardino 0.11143182972470737
San Bernardino 0.09971417702310345
Placer 0.20035576623590004
Placer 0.06404614010260831
Placer 0.16214478522225725
Placer 0.20220266798929115


In [36]:
df_2 = df.copy()
relevant3 = []
for i in pd.concat([df_tes4,df_tes5]).drop_duplicates(keep=False).values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    
    df_test3 = df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county")
    result = relative_risk(df_test3[df_test3.index == i[0]].values[0][0],df_test3[df_test3.index == i[0]].values[0][1]+df_test3[df_test3.index == i[0]].values[0][0],df_test3[df_test3.index != i[0]].values[0][0],df_test3[df_test3.index != i[0]].values[0][1]+df_test3[df_test3.index != i[0]].values[0][0])
    print(i[0], result.confidence_interval(confidence_level=0.99))
    
    # if result.confidence_interval(confidence_level=0.95)[0] > 1:
    #             relevant3.append(i)

Alpine ConfidenceInterval(low=0.9209753974300848, high=0.9616236830930587)
Lassen ConfidenceInterval(low=0.7213231405753907, high=0.8092827024165741)
Riverside ConfidenceInterval(low=0.9197931406663725, high=0.9877421097769051)
San Luis Obispo ConfidenceInterval(low=0.8629932203056192, high=0.9406011931851931)
Santa Clara ConfidenceInterval(low=0.8716810051212447, high=0.9185357079281524)
Placer ConfidenceInterval(low=0.8617769638713105, high=32.46901138386429)
San Bernardino ConfidenceInterval(low=0.8075714407499626, high=6.337073214979036)
San Bernardino ConfidenceInterval(low=0.8274774771628706, high=10.501320800406825)
San Bernardino ConfidenceInterval(low=0.7493989287642417, high=127.27507908875909)
San Bernardino ConfidenceInterval(low=0.7231397461560347, high=13.73535115879162)
San Bernardino ConfidenceInterval(low=0.7681935942702541, high=28.63326470916716)
Placer ConfidenceInterval(low=0.7770344651748091, high=133.02015666750913)
Placer ConfidenceInterval(low=0.831180012809226

In [28]:
df_tes4

,county,aisle,anzahl,rank,rank_mean
0,Alpine,baby food formula,2647,9.0,34.600000
1,Fresno,baby food formula,5425,12.0,34.600000
2,Glenn,baby food formula,6390,13.0,34.600000
3,Nevada,baby food formula,835,15.0,34.600000
4,Orange,baby food formula,6300,13.0,34.600000
...,...,...,...,...,...
195,Trinity,white wines,7,32.0,97.693878
196,Sierra,yogurt,1567,2.0,4.018182
197,Tehama,yogurt,718,1.0,4.018182
198,Yuba,yogurt,8243,2.0,4.018182


In [204]:
df_11 = df_2[df_2.aisle == "baby food formula"].groupby("county")[["aisle"]].count().reset_index()
merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == "baby food formula"].index))]
df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
df_test.loc[df_test.county != "Alpine",["aisle","not_aisle"]] = df_test.loc[df_test.county != "Alpine"].aisle.sum(),df_test.loc[df_test.county != "Alpine"].not_aisle.sum()
    
df_test3 = df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == "Alpine"))[-1]) | (df_test.county =="Alpine") ].set_index("county")
result = relative_risk(df_test3[df_test3.index == "Alpine"].values[0][1],df_test3[df_test3.index == "Alpine"].values[0][1]+df_test3[df_test3.index == "Alpine"].values[0][0],df_test3[df_test3.index != "Alpine"].values[0][1],df_test3[df_test3.index != "Alpine"].values[0][1]+df_test3[df_test3.index != "Alpine"].values[0][0])

  

In [205]:
result.confidence_interval(confidence_level=0.95)

ConfidenceInterval(low=0.9910290112401102, high=0.9925547102501968)

In [203]:
df_test3

,aisle,not_aisle
county,,
Alpine,2647,130636
Yuba,70476,5928504


In [30]:
df_test3.values[1,0]

1

In [31]:
result = relative_risk(df_test3.values[1,0],df_test3.values[1,1]+df_test3.values[1,0],df_test3.values[0,0],df_test3.values[0,0]+df_test3.values[0,1])

In [206]:
result.relative_risk

0.9917915673673409

In [34]:
result.confidence_interval(confidence_level=0.95)[0]

0.9478490483087167

In [171]:
df_9.head()

,county,aisle,anzahl,rank,rank_mean
29,Placer,air fresheners candles,1,36.0,108.600000
104,Trinity,asian foods,10,25.0,52.290909
212,Alpine,baby food formula,2647,9.0,34.600000
219,Fresno,baby food formula,5425,12.0,34.600000
220,Glenn,baby food formula,6390,13.0,34.600000


In [172]:
df_9.head().apply(lambda x: print(x),axis=1)

county                       Placer
aisle        air fresheners candles
anzahl                            1
rank                           36.0
rank_mean                     108.6
Name: 29, dtype: object
county           Trinity
aisle        asian foods
anzahl                10
rank                25.0
rank_mean      52.290909
Name: 104, dtype: object
county                  Alpine
aisle        baby food formula
anzahl                    2647
rank                       9.0
rank_mean                 34.6
Name: 212, dtype: object
county                  Fresno
aisle        baby food formula
anzahl                    5425
rank                      12.0
rank_mean                 34.6
Name: 219, dtype: object
county                   Glenn
aisle        baby food formula
anzahl                    6390
rank                      13.0
rank_mean                 34.6
Name: 220, dtype: object


29     None
104    None
212    None
219    None
220    None
dtype: object

In [176]:
df_2 = df.copy()

In [30]:
df_2

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


In [177]:
df_11 = df_2[df_2.aisle == "air fresheners candles"].groupby("county")[["aisle"]].count().reset_index()

In [178]:
df_11

,county,aisle
0,Alameda,6
1,Alpine,90
2,Amador,69
3,Calaveras,388
4,Colusa,19
5,Contra Costa,72
6,Del Norte,10
7,El Dorado,1
8,Fresno,143
9,Glenn,310


In [182]:
merged

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


In [149]:
merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index()

,county,not_aisle
0,Alameda,16910
1,Alpine,133193
2,Amador,121043
3,Calaveras,502091
4,Colusa,34924
5,Contra Costa,96649
6,Del Norte,19300
7,El Dorado,15422
8,Fresno,357798
9,Glenn,390104


In [150]:
df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())

In [153]:
df_test.loc[df_test.county != "Placer",["aisle","not_aisle"]] = df_test.loc[df_test.county != "Placer"].aisle.sum(),df_test.loc[df_test.county != "Placer"].not_aisle.sum()

In [160]:
df_test.loc[(df_test.county =="Alameda") | (df_test.county =="Placer") ]

,county,aisle,not_aisle
0,Alameda,3747,6127097
29,Placer,1,257


In [71]:
df_test = df_test.set_index("county")

KeyError: "None of ['county'] are in the columns"

#### des gleiche für departments

In [45]:
df_dep = df.groupby(["county","department"]).agg(anzahl=("department","count")).reset_index()

In [46]:
counties = df_dep.county.unique().tolist()
for i in counties:
    df_dep.loc[df_dep["county"] == i,"rank"] = df_dep.loc[df_dep["county"] == i].anzahl.rank(method="min",ascending=False)

In [160]:
df_dep

,county,department,anzahl,rank
0,Alameda,alcohol,170,15.0
1,Alameda,babies,40,16.0
2,Alameda,bakery,500,7.0
3,Alameda,beverages,2294,3.0
4,Alameda,breakfast,351,10.0
...,...,...,...,...
1168,Yuba,pantry,3244,10.0
1169,Yuba,personal care,3078,11.0
1170,Yuba,pets,552,18.0
1171,Yuba,produce,30046,1.0


In [47]:
df_dep1 = df_dep.groupby("department").agg(rank_mean=("rank","mean")).reset_index()

In [48]:
df_dep2 = pd.merge(df_dep,df_dep1)

In [49]:
df_dep3 = df_dep2.loc[(df_dep2["rank"] > df_dep2.rank_mean * 1.5)]
df_dep4 = df_dep2.loc[(df_dep2["rank"] < df_dep2.rank_mean * 0.5)]

In [50]:
df_dep3.__len__()

33

In [51]:
df_2 = df.copy()
relevant3 = []
for i in df_dep3.values:
    
    df_11 = df_2[df_2.department == i[1]].groupby("county")[["department"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.department == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_department=("department","count")).reset_index())
    df_test.loc[df_test.county != i[0],["department","not_department"]] = df_test.loc[df_test.county != i[0]].department.sum(),df_test.loc[df_test.county != i[0]].not_department.sum()
    
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county"))
    
    if p < 0.05:
        relevant3.append(i)

In [52]:
relevant3.__len__()

32

In [210]:
df_test

,county,department,not_department
0,Alameda,547538,5568751
1,Alpine,547538,5568751
2,Amador,547538,5568751
3,Calaveras,547538,5568751
4,Colusa,547538,5568751
5,Contra Costa,547538,5568751
6,Del Norte,547538,5568751
7,El Dorado,547538,5568751
8,Fresno,547538,5568751
9,Glenn,547538,5568751


In [53]:
df_2 = df.copy()
relevant4 = []
for i in df_dep3.values:
    
    df_11 = df_2[df_2.department == i[1]].groupby("county")[["department"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.department == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_department=("department","count")).reset_index())
    df_test.loc[df_test.county != i[0],["department","not_department"]] = df_test.loc[df_test.county != i[0]].department.sum(),df_test.loc[df_test.county != i[0]].not_department.sum()
    
    df_test3 = df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county")
    result = relative_risk(df_test3[df_test3.index == i[0]].values[0][0],df_test3[df_test3.index == i[0]].values[0][1]+df_test3[df_test3.index == i[0]].values[0][0],df_test3[df_test3.index != i[0]].values[0][0],df_test3[df_test3.index != i[0]].values[0][1]+df_test3[df_test3.index != i[0]].values[0][0])
    if result.confidence_interval(confidence_level=0.99)[0] > 1:
            relevant4.append(i)
  

confidence intervall spatlen und zeilen vertauschen

In [54]:
relevant4.__len__()

0

In [178]:
df_dep3.head()

,county,department,anzahl,rank,rank_mean
118,Humboldt,bakery,119,12.0,7.754386
156,Stanislaus,bakery,615,12.0,7.754386
192,Nevada,beverages,2051,8.0,4.070175
201,San Francisco,beverages,2138,13.0,4.070175
202,San Joaquin,beverages,280,10.0,4.070175


In [179]:
df_11 = df_2[df_2.department == "bakery"].groupby("county")[["department"]].count().reset_index()
merged = df_2[~(df_2.index.isin(df_2[df_2.department == "bakery"].index))]
df_test = pd.merge(df_11,merged.groupby("county").agg(not_department=("department","count")).reset_index())
df_test.loc[df_test.county != "Humboldt",["department","not_department"]] = df_test.loc[df_test.county != "Humboldt"].department.sum(),df_test.loc[df_test.county != "Humboldt"].department.sum()
    
df_test3 = df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == "Humboldt"))[-1]) | (df_test.county =="Humboldt") ].set_index("county")

In [180]:
df_test3

,department,not_department
county,,
Humboldt,119,6282
Yuba,223524,223524


In [185]:
df_test3[df_test3.index == "Humboldt"].values[0][1]

6282

In [186]:
result = relative_risk(df_test3[df_test3.index == "Humboldt"].values[0][1],df_test3[df_test3.index == "Humboldt"].values[0][1]+df_test3[df_test3.index == "Humboldt"].values[0][0],df_test3[df_test3.index != "Humboldt"].values[0][1],df_test3[df_test3.index != "Humboldt"].values[0][1]+df_test3[df_test3.index != "Humboldt"].values[0][0])

In [187]:
result.confidence_interval(confidence_level=0.95)

ConfidenceInterval(low=1.9540683785979154, high=1.9716074211379073)

#### gleiche für produkte

In [55]:
df_pro = df.groupby(["county","product_name"]).agg(anzahl=("product_name","count")).reset_index()
counties = df_pro.county.unique().tolist()
for i in counties:
    df_pro.loc[df_pro["county"] == i,"rank"] = df_pro.loc[df_pro["county"] == i].anzahl.rank(method="min",ascending=False)

In [56]:
df_pro

,county,product_name,anzahl,rank
0,Alameda,0% Fat Free Organic Milk,3,993.0
1,Alameda,0% Greek Strained Yogurt,120,8.0
2,Alameda,1% Low Fat Milk,7,457.0
3,Alameda,1% Milkfat Low Fat Vitamin A & D Milk,1,1930.0
4,Alameda,100 Calorie Per Bag Popcorn,12,259.0
...,...,...,...,...
579229,Yuba,with Olive Oil Mayonnaise,1,10261.0
579230,Yuba,with Olive Oil Mayonnaise Dressing,1,10261.0
579231,Yuba,with Twist Ties Sandwich & Storage Bags,4,5618.0
579232,Yuba,with Xylitol Original Flavor 18 Sticks Sugar F...,6,4318.0


In [60]:
df_pro1 = df_pro.groupby("product_name").agg(rank_mean=("rank","mean")).reset_index()
df_pro2 = pd.merge(df_pro,df_pro1)
df_pro3 = df_pro2.loc[(df_pro2["rank"] > df_pro2.rank_mean * 1.5)]
df_pro4 = df_pro2.loc[(df_pro2["rank"] < df_pro2.rank_mean * 0.5)]

In [62]:
df_pro3[df_pro3.anzahl > 30]

,county,product_name,anzahl,rank,rank_mean
45,Calaveras,0% Greek Strained Yogurt,40,2295.0,1153.121951
131,Ventura,1% Low Fat Milk,31,1582.0,987.632653
1072,Modoc,100% Raw Coconut Water,34,493.0,257.641509
1406,Sacramento,100% Recycled Paper Towels,32,507.0,304.153846
1573,Colusa,100% Whole Wheat Bread,38,136.0,86.836364
...,...,...,...,...,...
302319,Glenn,Peanut Butter Creamy With Salt,33,2313.0,1322.000000
352173,Calaveras,Traditional Favorites Tomato & Basil Pasta Sauce,34,2696.0,1680.244444
366214,Alpine,White Onion,37,663.0,428.547170
366230,Los Angeles,White Onion,42,676.0,428.547170


In [65]:
df_pro4[df_pro4.anzahl > 30]

,county,product_name,anzahl,rank,rank_mean
14,Los Angeles,0% Fat Free Organic Milk,43,654.0,2370.139535
17,Mariposa,0% Fat Free Organic Milk,46,900.0,2370.139535
32,Santa Cruz,0% Fat Free Organic Milk,88,269.0,2370.139535
40,Ventura,0% Fat Free Organic Milk,43,1087.0,2370.139535
43,Alameda,0% Greek Strained Yogurt,120,8.0,1153.121951
...,...,...,...,...,...
566668,Modoc,Simply Plain Yogurt Protein Drink With Crunchy...,31,551.0,7514.600000
570227,Marin,No Sugar Added Lemon & Cherry Flavor Real Ital...,31,403.0,6536.000000
570725,Ventura,Blanched Unsalted Hazelnuts (Filberts),35,1376.0,6152.666667
573307,Yolo,"Milk, Goat",53,194.0,2578.333333


In [66]:
df_2 = df.copy()
relevant5 = []
for i in df_pro3[df_pro3.anzahl > 30].values:
    
    df_11 = df_2[df_2.product_name == i[1]].groupby("county")[["product_name"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.product_name == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_product_name=("product_name","count")).reset_index())
    df_test.loc[df_test.county != i[0],["product_name","not_product_name"]] = df_test.loc[df_test.county != i[0]].product_name.sum(),df_test.loc[df_test.county != i[0]].not_product_name.sum()
    
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county"))
    
    if p < 0.05:
        relevant5.append(i)